## Motion Planning through KOMO ##

In [1]:
import robotic as ry
import numpy as np
import time

## Part a) Adding a constraint to simulation

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

# add waypoints
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way2'). setShape(ry.ST.marker, [.1]) .setPosition([0, .2, 1.6])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

# add wall
C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()

# save home joints state
qHome = C.getJointState()

First, solve the task without collision constraint

In [3]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)

# order=0: penalizing sqr distance to qHome, 
# order=1: penalizing sqr distances between consecutive configurations (velocities), 
# order=2: penalizing accelerations across 3 configurations
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)

# add accumuated colisions penalty
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)


# add waypoints as objectives
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([2], ry.FS.positionDiff, ['l_gripper', 'way2'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([4], ry.FS.positionDiff, ['l_gripper', 'way2'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

# get the initial path using Non Linear Programming solver
ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)

# get the optimized path. q is a matrix of joint states
q = komo.getPath()
print(q)

# simulate the optimized path
for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.315941, evals: 219, done: 1, feasible: 1, sos: 5.65488, f: 0, ineq: 0, eq: 0.140218 }
[[-0.03424207 -0.44964337 -0.03981783 -2.001002   -0.01271247  2.00646179
  -0.4975922 ]
 [-0.0685065  -0.39922657 -0.07968621 -2.00199007 -0.02543919  2.01290744
  -0.49517654]
 [-0.10281562 -0.3486893  -0.11965578 -2.00295023 -0.03819454  2.01932076
  -0.49274516]
 [-0.13719158 -0.29797113 -0.15977745 -2.00386841 -0.05099307  2.0256854
  -0.49029014]
 [-0.17165637 -0.24701135 -0.20010244 -2.00473039 -0.06384967  2.03198478
  -0.48780355]
 [-0.20623164 -0.19574888 -0.24068249 -2.00552171 -0.07677969  2.038202
  -0.4852774 ]
 [-0.24093867 -0.14412213 -0.28156992 -2.00622762 -0.0897991   2.04431967
  -0.48270363]
 [-0.27579821 -0.09206893 -0.32281785 -2.00683292 -0.10292465  2.05031978
  -0.48007414]
 [-0.31083037 -0.03952633 -0.3644803  -2.00732188 -0.11617414  2.05618345
  -0.47738071]
 [-0.34605438  0.01356956 -0.40661241 -2.00767803 -0.12956669  2.0618907
  -0.47461507]
 [-0.30056378 -0.0

In [4]:
del komo
del C

Then, we add collision constraint with "accumulatedCollisions" feature

In [5]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

But we see that there is a jump in the motion while passing the obstacle

Here we will see the importance of the scale of the constraint (give values 0, 100 and 1 (which is default one) relatively). Then put 0.1 (1e-1) for perfect case. Do not forget to Del K and C or restart.

In [6]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.191114, evals: 61, done: 1, feasible: 0, sos: 3.33824, f: 0, ineq: 0, eq: 0.793235 }
[[-0.03630787 -0.45150972 -0.03923283 -2.0036983  -0.00710369  2.01073889
  -0.49810271]
 [-0.07264936 -0.40295808 -0.07850626 -2.00738145 -0.01421965  2.02146223
  -0.4961971 ]
 [-0.109058   -0.35428378 -0.11786097 -2.01103428 -0.02136028  2.03215434
  -0.49427496]
 [-0.14556718 -0.30542551 -0.1573378  -2.01464164 -0.02853821  2.04279924
  -0.49232831]
 [-0.18221006 -0.25632204 -0.19697787 -2.01818837 -0.03576646  2.0533805
  -0.49034949]
 [-0.21901951 -0.20691218 -0.23682262 -2.02165933 -0.04305849  2.06388119
  -0.48833129]
 [-0.25602808 -0.15713469 -0.27691395 -2.02503933 -0.05042838  2.07428377
  -0.48626709]
 [-0.29326789 -0.10692827 -0.31729431 -2.02831312 -0.05789085  2.08457004
  -0.48415096]
 [-0.33077068 -0.05623145 -0.3580068  -2.03146541 -0.06546142  2.09472116
  -0.48197781]
 [-0.3685677  -0.00498248 -0.3990953  -2.03448078 -0.07315642  2.10471767
  -0.47974352]
 [-0.33633061 -0

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);

    komo.addObjective(...): This is calling a method named addObjective on the komo object, which is an instance of a KOMO optimizer.

    []: The first argument [] specifies the time intervals over which this objective applies. In this case, [] means it applies to all time intervals. In other words, this objective is valid for the entire duration of the optimization.

    ry.FS.accumulatedCollisions: The second argument ry.FS.accumulatedCollisions is specifying the type of feature to use for this objective. In this case, it's using a feature related to accumulated collisions. This means the objective will aim to minimize the total amount of collisions that occur throughout the motion.

    []: The third argument [] specifies the frames associated with this feature. Since ry.FS.accumulatedCollisions doesn't require specific frame associations, this argument is left empty.

    ry.OT.eq: The fourth argument ry.OT.eq specifies the type of objective. Here, it's an equality constraint. This means the optimizer will aim to make the value of the accumulated collisions equal to a certain target value (which is default value zero in here).

Putting it all together, this line is instructing the optimizer to minimize the total amount of accumulated collisions over the entire duration of the motion, treating it as an equality constraint. This is a way to prioritize collision-free or low-collision solutions.

##  Part b) Possible State Solution by KOMO

In [7]:
del komo
del C

In [8]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.0])

C.addFrame('box') \
    .setPosition([-.25,.1,1.3]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.5,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [9]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)

komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])

komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.289301, evals: 698, done: 1, feasible: 1, sos: 61.1134, f: 0, ineq: 0, eq: 0.135414 }
size of path: (10, 7)


## Possible State Solution by RRT

In [30]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [31]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_panda_coll5) [64,51] 	d=-0
{ time: 0.0055, evals: 77, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


In [32]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [33]:
del komo
del rrt
del C

Now, let us give a state which is impossible for robot arm to reach.

##  Part c) Impossible State Solution by KOMO

In [34]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.7,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [35]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)
for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.142184, evals: 292, done: 1, feasible: 0, sos: 28.8058, f: 0, ineq: 0, eq: 2.87005 }
size of path: (10, 7)


## Impossible State Solution by RRT

In [36]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [37]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

-- RRT_PathFinder.cpp:RRT_PathFinder:257(0) initializing with infeasible q0:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_finger1) [64,58] 	d=-0
proxy:  (box)-(table) [64,1] 	d=-0
proxy:  (box)-(l_finger2) [64,59] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_finger1) [64,58] 	d=-0
proxy:  (box)-(table) [64,1] 	d=-0
proxy:  (box)-(l_finger2) [64,59] 	d=-0
{ time: 0.166388, evals: 5001, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }


So we will get an error since there is no feasible solution

In [38]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

IndexError: tuple index out of range

In [41]:
# https://marctoussaint.github.io/robotics-course/tutorials.html -> Tutorials
# https://github.com/MarcToussaint/rai-tutorials/tree/main -> Tutorials
# https://marctoussaint.github.io/robotics-course/rai.html -> Documentation